In [1]:
import os

os.chdir("../..")
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import copy
from collections import Counter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm.auto as tqdm

from modular_splicing.dataset.datafile_object import SpliceAIDatafile

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from modular_splicing.case_study.yang.pipeline import asb_table, exons_table

In [5]:
asb = asb_table()

In [6]:
exons = exons_table()

In [7]:
files = [SpliceAIDatafile.load(path) for path in ("datafile_train_all.h5", "datafile_test_0.h5")]
def id_gene(gene_name):
    for i in range(2):
        names = files[i].names.tolist()
        if gene_name not in names:
            continue
        idx = names.index(gene_name)
        return (
            files[i].chroms[idx],
            files[i].strands[idx],
            int(files[i].datafile["TX_START"][idx]),
            int(files[i].datafile["TX_END"][idx]),
        )
    raise RuntimeError

In [8]:
def get_asb_for(gene_name, rbp_name, major, minor):
    gene_chrom, gene_strand, gene_start, gene_end = id_gene(gene_name)
    print(gene_strand)
    assert gene_start <= gene_end
    return asb[
        (asb.Chr == gene_chrom)
        & (asb.strand == gene_strand)
        & (gene_start <= asb.Coordinate)
        & (asb.Coordinate <= gene_end)
        & (asb.RBP == rbp_name)
        & (asb["Major allele"] == major_minor[0])
        & (asb["Minor allele"] == major_minor[1])
    ]

def get_exons_for(gene_name, rbp_name):
    gene_chrom, gene_strand, gene_start, gene_end = id_gene(gene_name)
    print(gene_strand)
    assert gene_start <= gene_end
    return exons[
        (exons.chr == gene_chrom)
        & (exons.strand == gene_strand)
        & (gene_start <= exons.start)
        & (exons.end <= gene_end)
        & (exons.RBP == rbp_name)
    ]

In [9]:
asb

1,RBP,SNV ID,Chr,Coordinate,strand,Read counts,Major allele,Minor allele,P value,FDR
2,AGGF1,chr3_65922590_-,chr3,65922590,-,"A:0,C:0,G:0,T:26,_A:0,C:0,G:1,T:11,",T,G,0.02456,0.094875
3,AGGF1,chr11_2152005_-,chr11,2152005,-,"A:3773,C:1,G:3,T:2,_A:3179,C:0,G:2,T:1,",A,C,0.010501,0.062815
4,AGGF1,chr20_62324724_+,chr20,62324724,+,"A:0,C:84,G:0,T:51,_A:0,C:84,G:0,T:61,",C,T,0.025641,0.097076
5,AGGF1,chr21_46848841_+,chr21,46848841,+,"A:0,C:9,G:0,T:0,_A:0,C:19,G:0,T:0,",C,T,0.022445,0.090453
6,AKAP1,chr1_230416399_+,chr1,230416399,+,"A:0,C:1,G:33,T:13,_A:0,C:0,G:22,T:8,",G,T,0.020328,0.085886
...,...,...,...,...,...,...,...,...,...,...
3703,YBX3,chr2_241467180_-,chr2,241467180,-,"A:29,C:0,G:11,T:0,_A:29,C:0,G:14,T:0,",A,G,0.013712,0.077289
3704,YBX3,chr16_81611849_+,chr16,81611849,+,"A:120,C:0,G:61,T:0,_A:288,C:0,G:126,T:0,",A,G,0.003894,0.053114
3705,YBX3,chr16_81611851_+,chr16,81611851,+,"A:1,C:62,G:0,T:99,_A:0,C:131,G:0,T:249,",T,C,0.01097,0.071511
3706,ZNF800,chr1_16976641_+,chr1,16976641,+,"A:0,C:17,G:0,T:6,_A:0,C:16,G:0,T:2,",C,T,0.003937,0.053303


In [10]:
exons

,exon,cell,RBP,samp,dPSI,chr,start,end,strand
0,chr4:1219148:1219354:-,K562,AGGF1,ASB,23.0,chr4,1219148,1219354,-
1,chr21:47612518:47613088:-,K562,AGGF1,ASB,0.1,chr21,47612518,47613088,-
2,chr4:1219148:1219354:-,K562,AGGF1,ASB,23.0,chr4,1219148,1219354,-
3,chr21:47612518:47613088:-,K562,AGGF1,ASB,0.1,chr21,47612518,47613088,-
4,chr21:40574251:40574471:-,K562,KHSRP,ASB,-1.3,chr21,40574251,40574471,-
...,...,...,...,...,...,...,...,...,...
1645086,chrX:84499038:84499197:+,K562,ZRANB2,ctrl,-11.2,chrX,84499038,84499197,+
1645087,chrX:84499090:84499197:+,K562,ZRANB2,ctrl,0.0,chrX,84499090,84499197,+
1645088,chrX:84499101:84499177:+,K562,ZRANB2,ctrl,0.0,chrX,84499101,84499177,+
1645089,chrX:85158641:85158712:-,K562,ZRANB2,ctrl,0.0,chrX,85158641,85158712,-


In [11]:
get_asb_for("MAP2K3", "BUD13", "C", "G")

+


NameError: name 'major_minor' is not defined

In [ ]:
get_exons_for("MAP2K3", "BUD13")

In [ ]:
from modular_splicing.motif_names import get_motif_names

In [ ]:
in_figures = {"BUD13", "PRPF4", "SF3A3", "SF3B3", "MTPAP", "ZNF622", "U2AF2", "PRPF8", "UPF1"}
assert not in_figures & set(get_motif_names("rbns"))
assert not in_figures & set(get_motif_names("rbns_v2p1"))

In [ ]:
21208349 - 21208363, 21208349 - 21208440

In [ ]:
get_asb_for("MAP2K3", "PRPF4", "C", "G")

In [ ]:
get_exons_for("MAP2K3", "PRPF4")

In [ ]:
exons[(exons.RBP == "PRPF4")]

In [ ]:
get_asb_for("MAP2K3", "SF3A3", "C", "G")

In [ ]:
get_exons_for("MAP2K3", "SF3A3")

In [ ]:
get_asb_for("MAP2K3", "SF3B4", "C", "G")

In [ ]:
get_exons_for("MAP2K3", "SF3B4")

In [ ]:
get_asb_for("LARS", "U2AF2", "C", "U")